# Home work "CTC-loss"

Задача: реализовать прямой (forward) проход и вчислить матрицу $\alpha_t(s)$. Значения этой матрицы должны равняться
- $\alpha_t(s)$ в случае если валидные пути проходят через данную ячейку
- 0.0 в противном случае

Размерности входов и результата см. по коду.

Ноутбук с решением присыслать на voropaev@corp.mail.ru Тему письма пишите пожалуйста в формате "[МИФИ] Фамилия"

Deadline: 21.12.2018

In [1]:
import torch

In [4]:
# Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,2,3,1,4,5,4,3,2,1,], dtype=torch.long)

# y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1,  0,  0., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0,  0., 1., 0., 0., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 0., 0., 1.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],], dtype=torch.float32)

In [58]:
def compute_ctc_alpha(l, y):
    """
    Функция, вычисляющая матрицу $\alpha$ для данного входа.
    @param l метки символов строки, размерностью [L,]
    @param y предсказанные сетью вероятности для каждого фрейма. Размерность [T, Lexicon+1]
    @return матрицу $\alpha$ размерностью [2*L+1, T]
    """
    
    #TODO Вписать решение сюда. 
    W = torch.zeros((2 * l.size()[0] + 1), dtype=torch.int)
    for i, j in zip(range(1, 2 * l.size()[0], 2), range(l.size()[0])):
        W[i] = l[j]
    M = torch.t(y)
    
    result = torch.zeros((2 * l.size()[0] + 1, y.size()[0]))
    result[0, 0] = M[W[0], 0]
    result[0, 1] = M[W[1], 0]

    for s in range(2 * l.size()[0] + 1):
        for  t in range(1, y.size()[0]):
            if W[s] == 0 :
                result[s, t] = (result[s - 1, t - 1] + result[s, t - 1]) * M[W[s], t]
            elif W[s] == W[s - 2]:
                result[s, t] = (result[s - 1, t - 1] + result[s, t - 1]) * M[W[s], t]
            else:
                result[s, t] = (result[s - 2, t - 1] + result[s - 1, t - 1] + result[s, t - 1]) * M[W[s], t]
    return result

In [59]:
compute_ctc_alpha(l, y)

tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0

In [61]:
# Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha
def test():
    al = compute_ctc_alpha(l, y)
    ritght_al = torch.tensor([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    
    if torch.all(al == ritght_al):
        return True
    else:
        return False
    
assert test(), "Test faled"
    